In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import spherical_jn, spherical_yn
from scipy.constants import pi
from IPython.display import display, clear_output
import ipywidgets as widgets

# Function to calculate Mie coefficients
def mie_coefficients(x, m, n_max):
    a_n = np.zeros(n_max, dtype=complex)
    b_n = np.zeros(n_max, dtype=complex)

    for n in range(1, n_max + 1):
        a_n[n - 1] = (m * spherical_jn(n, x) - spherical_jn(n, x, True)) / \
                     (m * spherical_yn(n, x) - spherical_yn(n, x, True))
        b_n[n - 1] = (spherical_jn(n, x) - m * spherical_jn(n, x, True)) / \
                     (spherical_yn(n, x) - m * spherical_yn(n, x, True))

    return a_n, b_n

# Function to calculate Mie scattering intensity
def mie_scattering(x, m, theta, polarization):
    n_max = 100  # Maximum order of the Mie series

    a_n, b_n = mie_coefficients(x, m, n_max)

    if polarization == 'parallel':
        polarization_factor = np.abs(np.cos(theta))**2
    elif polarization == 'perpendicular':
        polarization_factor = np.abs(np.sin(theta))**2
    elif polarization == 'unpolarized':
        polarization_factor = 0.5 * (np.abs(np.cos(theta))**2 + np.abs(np.sin(theta))**2)

    intensity = polarization_factor * np.sum((2 * np.arange(1, n_max + 1) + 1) *
                                             (np.abs(a_n)**2 + np.abs(b_n)**2))

    return intensity

# Function to calculate and display Mie scattering results
def calculate_and_display(lambda_nm, I_0, r, n, d, intensity_scale):
    # Convert wavelength from nanometers to meters
    lambda_m = lambda_nm * 1e-9

    # Calculate size parameter
    x = (2 * pi * r) / lambda_m

    # Scattering angles in degrees (from 0 to 180 degrees) with a step size of 0.1
    theta_degrees = np.arange(0, 180.1, 0.1)

    # Convert scattering angles from degrees to radians
    theta_radians = np.deg2rad(theta_degrees)

    # Calculate the Mie scattering intensity for each scattering angle
    intensity_parallel = [mie_scattering(x, n, theta, 'parallel') for theta in theta_radians]
    intensity_perpendicular = [mie_scattering(x, n, theta, 'perpendicular') for theta in theta_radians]
    intensity_unpolarized = [mie_scattering(x, n, theta, 'unpolarized') for theta in theta_radians]

    # Create a DataFrame
    df1 = pd.DataFrame({
        'angle': theta_degrees,
        'parallel': intensity_parallel,
        'perpendicular': intensity_perpendicular,
        'unpolarized': intensity_unpolarized
    })
    global df
    df = df1.copy()

    # Display the DataFrame
    display(df)

    # Plot the graph
    plt.figure(figsize=(10, 6))
    plt.style.use('ggplot')

    if intensity_scale == 'linear':
        plt.plot(theta_degrees, intensity_parallel, color='red', label='Parallel Polarization')
        plt.plot(theta_degrees, intensity_perpendicular, color='red', linestyle='--', label='Perpendicular Polarization')
        plt.plot(theta_degrees, intensity_unpolarized, color='blue', label='Unpolarized')
        plt.ylabel('Intensity (linear scale)', color='black')
    elif intensity_scale == 'log':
        plt.semilogy(theta_degrees, intensity_parallel, color='red', label='Parallel Polarization')
        plt.semilogy(theta_degrees, intensity_perpendicular, color='red', linestyle='--', label='Perpendicular Polarization')
        plt.semilogy(theta_degrees, intensity_unpolarized, color='blue', label='Unpolarized')
        plt.ylabel('Intensity (logarithmic scale)', color='black')

    plt.xlabel('Scattering Angle (degrees)', color='black')
    plt.xlim(0, 180)
    plt.xticks(np.arange(0, 181, 30))
    plt.title('Mie Scattering Intensity vs Scattering Angle', fontweight='bold', fontname='Franklin Gothic Medium')

    columnLabels = ['Input Parameters', 'Values']
    tableValues = [['λ', f' {lambda_nm} nm'], ['Radius of particle(r)', f'{r*1e6} μm'], ['Ref. Index(n)', n],['Observer Distance', f'{d} m'], ['Incident Intensity I\u2080', f'{I_0} (W/m\u00B2)']]
    my_table = plt.table(cellText=tableValues, colWidths=[0.17, 0.1], colLabels=columnLabels, loc='lower right')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.7)
    plt.show()

    return df

# Create button to trigger update
update_button = widgets.Button(description="Update Values")
output = widgets.Output()

# Define the update function
def update_values(_):
    with output:
        clear_output(wait=True)
        lambda_nm_val = lambda_nm.value
        I_0_val = I_0.value
        r_val = r.value * 1e-6
        n_val = n.value
        d_val = d.value
        intensity_scale_val = intensity_scale.value

        result_df = calculate_and_display(lambda_nm_val, I_0_val, r_val, n_val, d_val, intensity_scale_val)

        # Now you can access 'result_df' outside this function

# Set initial values
lambda_nm = widgets.IntText(value=650, min=400, max=700, step=1, description='Wavelength (nm):')
I_0 = widgets.FloatText(value=1, description='Intensity (W/m²):')
r = widgets.FloatSlider(value=0.01, min=0, max=1, step=0.01, description='Particle Radius (μm):')
n = widgets.FloatText(value=1.5, description='Refractive Index:')
d = widgets.FloatText(value=1, description='Observer Distance (m):')
intensity_scale = widgets.Dropdown(options=['linear', 'log'], value='linear', description='Intensity Scale:')

# Attach the update function to the button click event
update_button.on_click(update_values)

# Display widgets and button
display(widgets.VBox([widgets.Box([lambda_nm]), widgets.Box([I_0]), widgets.Box([r]), widgets.Box([n]), widgets.Box([d]), widgets.Box([intensity_scale]), update_button, output]))


In [ ]:
df.shape

NameError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame with columns 'angle', 'parallel', 'perpendicular', 'unpolarized'
X = df[['angle']]  # Features
y = df[['parallel', 'perpendicular', 'unpolarized']]  # Target variables

# Split the data into training and testing sets (80:20 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt

# Plot angle vs. perpendicular polarization
plt.figure(figsize=(10, 6))
plt.plot(df['angle'], df['perpendicular'], color='green', label='Perpendicular Polarization')
plt.xlabel('Scattering Angle (degrees)')
plt.ylabel('Intensity')
plt.title('Angle vs. Perpendicular Polarization Intensity')
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.show()

# Plot angle vs. parallel polarization
plt.figure(figsize=(10, 6))
plt.plot(df['angle'], df['parallel'], color='red', label='Parallel Polarization')
plt.xlabel('Scattering Angle (degrees)')
plt.ylabel('Intensity')
plt.title('Angle vs. Parallel Polarization Intensity')
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.show()

# Plot angle vs. unpolarized intensity
plt.figure(figsize=(10, 6))
plt.plot(df['angle'], df['unpolarized'], color='blue', label='Unpolarized')
plt.xlabel('Scattering Angle (degrees)')
plt.ylabel('Intensity')
plt.title('Angle vs. Unpolarized Intensity')
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.show()


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Assuming df is your DataFrame containing Mie scattering data
# If not, replace df with your actual DataFrame

# Selecting the angle as the input feature and all three intensities as target variables
X = df['angle'].values.reshape(-1, 1)
y_parallel = df['parallel'].values.reshape(-1, 1)
y_perpendicular = df['perpendicular'].values.reshape(-1, 1)
y_unpolarized = df['unpolarized'].values.reshape(-1, 1)

# Split the data into training and testing sets
X_train, X_test, y_train_parallel, y_test_parallel, y_train_perpendicular, y_test_perpendicular, y_train_unpolarized, y_test_unpolarized = train_test_split(
    X, y_parallel, y_perpendicular, y_unpolarized, test_size=0.2, random_state=42
)

# Define the kernel for Gaussian process regression
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

# Create separate Gaussian process regressors for parallel, perpendicular, and unpolarized intensities
gp_parallel = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp_perpendicular = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp_unpolarized = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Fit the models
gp_parallel.fit(X_train, y_train_parallel)
gp_perpendicular.fit(X_train, y_train_perpendicular)
gp_unpolarized.fit(X_train, y_train_unpolarized)

# Predict on the test data
y_pred_parallel, sigma_parallel = gp_parallel.predict(X_test, return_std=True)
y_pred_perpendicular, sigma_perpendicular = gp_perpendicular.predict(X_test, return_std=True)
y_pred_unpolarized, sigma_unpolarized = gp_unpolarized.predict(X_test, return_std=True)

# Plot the results for parallel intensity
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test_parallel, color='black', label='Actual Data (Parallel)')
plt.plot(X_test, y_pred_parallel, color='blue', label='GP Prediction (Parallel)')
plt.fill_between(X_test.flatten(), y_pred_parallel.flatten() - sigma_parallel, y_pred_parallel.flatten() + sigma_parallel, alpha=0.2, color='blue')
plt.xlabel('Scattering Angle (degrees)')
plt.ylabel('Intensity (Parallel)')
plt.title('Gaussian Process Regression for Mie Scattering (Parallel)')
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.show()

# Plot the results for perpendicular intensity
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test_perpendicular, color='black', label='Actual Data (Perpendicular)')
plt.plot(X_test, y_pred_perpendicular, color='red', label='GP Prediction (Perpendicular)')
plt.fill_between(X_test.flatten(), y_pred_perpendicular.flatten() - sigma_perpendicular, y_pred_perpendicular.flatten() + sigma_perpendicular, alpha=0.2, color='red')
plt.xlabel('Scattering Angle (degrees)')
plt.ylabel('Intensity (Perpendicular)')
plt.title('Gaussian Process Regression for Mie Scattering (Perpendicular)')
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.show()

# Plot the results for unpolarized intensity
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test_unpolarized, color='black', label='Actual Data (Unpolarized)')
plt.plot(X_test, y_pred_unpolarized, color='green', label='GP Prediction (Unpolarized)')
plt.fill_between(X_test.flatten(), y_pred_unpolarized.flatten() - sigma_unpolarized, y_pred_unpolarized.flatten() + sigma_unpolarized, alpha=0.2, color='green')
plt.xlabel('Scattering Angle (degrees)')
plt.ylabel('Intensity (Unpolarized)')
plt.title('Gaussian Process Regression for Mie Scattering (Unpolarized)')
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.show()


In [ ]:
# Predict on the test data
y_pred_parallel, sigma_parallel = gp_parallel.predict(X_test, return_std=True)
y_pred_perpendicular, sigma_perpendicular = gp_perpendicular.predict(X_test, return_std=True)
y_pred_unpolarized, sigma_unpolarized = gp_unpolarized.predict(X_test, return_std=True)

# # Plot the results for parallel intensity
# plt.scatter(X_test, y_test_parallel, color='black', label='Actual Data (Parallel)')
# plt.plot(X_test, y_pred_parallel, color='blue', label='GP Prediction (Parallel)')
# plt.fill_between(X_test.flatten(), y_pred_parallel.flatten() - sigma_parallel, y_pred_parallel.flatten() + sigma_parallel, alpha=0.2, color='blue')
# # ... (similar plots for perpendicular and unpolarized intensities)


In [ ]:
from sklearn.metrics import r2_score

# Assuming you have y_test_unpolarized and y_pred_unpolarized from your Mie scattering code
# Update variable names accordingly if needed
r2_unpolarized = r2_score(y_test_unpolarized, y_pred_unpolarized)
print(f"R-squared (R2) Score for Unpolarized Intensity: {int(r2_unpolarized * 100)} %")


In [ ]:
from sklearn.metrics import mean_squared_error

# Assuming you have y_test_parallel and y_pred_parallel from your Mie scattering code
# Update variable names accordingly if needed
mse_parallel = mean_squared_error(y_test_parallel, y_pred_parallel)
print(f"Mean Squared Error (MSE) for Parallel Intensity: {mse_parallel}")

# Assuming you have y_test_perpendicular and y_pred_perpendicular from your Mie scattering code
# Update variable names accordingly if needed
mse_perpendicular = mean_squared_error(y_test_perpendicular, y_pred_perpendicular)
print(f"Mean Squared Error (MSE) for Perpendicular Intensity: {mse_perpendicular}")

# Assuming you have y_test_unpolarized and y_pred_unpolarized from your Mie scattering code
# Update variable names accordingly if needed
mse_unpolarized = mean_squared_error(y_test_unpolarized, y_pred_unpolarized)
print(f"Mean Squared Error (MSE) for Unpolarized Intensity: {mse_unpolarized}")



In [ ]:
custom_value = float(input("Enter the angle value: "))  # Taking the custom value for prediction

# Assuming 'gp_parallel' is the Gaussian Process Regressor for parallel intensity
predicted_value_parallel, _ = gp_parallel.predict([[custom_value]], return_std=True)

print(f"The predicted value of parallel light intensity is: {predicted_value_parallel[0]}")  # No indexing needed

# Assuming 'gp_unpolarized' is the Gaussian Process Regressor for unpolarized intensity
predicted_value_unpolarized, _ = gp_unpolarized.predict([[custom_value]], return_std=True)

print(f"The predicted value of unpolarized light intensity is: {predicted_value_unpolarized[0]}")  # No indexing needed

# Assuming 'gp_perpendicular' is the Gaussian Process Regressor for perpendicular intensity
predicted_value_perpendicular, _ = gp_perpendicular.predict([[custom_value]], return_std=True)

print(f"The predicted value of perpendicular light intensity is: {predicted_value_perpendicular[0]}")  # No indexing needed


In [ ]:
l = X_test.shape[0] - 1

def slope(m, n):
    xn = []
    yn = []
    sl = []

    for i in range(0, n + 1):
        index = int(((l / n) * i) + 1)
        if index == X_test.shape[0]:
            index = X_test.shape[0] - 1
        xn.append(X_test[index][0])  # Assuming X_test is your test data
        yn.append(y_test_parallel[index][0])  # Assuming y_test_parallel is your test data for parallel intensity

    plt.plot(xn, yn, marker='s', linestyle='--')

    for i in range(0, n):
        sl.append((yn[i + 1] - yn[i]) / (xn[i + 1] - xn[i]))

    min_slope = np.min(sl)
    max_slope = np.max(sl)

    if m > min_slope and m < max_slope:
        diff_from_m = np.abs(np.array(sl) - m)
        minpos = np.argmin(diff_from_m)

        print("-----------The slope for the various divisions in the graph are -------------\n")
        for i in sl:
            print(i)

        print("-----------The slope that matches the input in the graph are -------------\n")
        print(sl[minpos], " at position ", minpos + 1)

        plt.show()

    else:
        print("The slope you have entered is not possible for the given particle\n")

def intensity(angle):
    prdicted_value_parallel, _ = gp_parallel.predict([[angle]], return_std=True)
    return prdicted_value_parallel[0][0]

def slopeCompare():
    n = int(input("Enter the number of divisions of the graph \n"))
    print("Select a mode: \n1.Slope \n2.Angle (single) \n3.Angle (dual)\n4.Slope (dual)")
    x = int(input("Enter mode (1,2,3,4):\n"))

    if x == 1:
        m = float(input("Enter slope: \n"))
        slope(m, n)
    elif x == 2:
        angle = float(input("Enter angle: \n"))
        m = (intensity(angle + 0.001) - intensity(angle - 0.001)) / (0.002)
        print("Slope we are looking for is: ", m)
        slope(m, n)
    elif x == 3:
        angle1 = float(input("Enter larger angle: \n"))
        angle2 = float(input("Enter smaller angle: \n"))
        m = (intensity(angle1) - intensity(angle2)) / (angle1 - angle2)
        print("Slope we are looking for is: ", m)
        slope(m, n)
    elif x == 4:
        s1 = float(input("Enter slope 1: \n"))
        s2 = float(input("Enter slope 2: \n"))
        slope((s1 + s2) / 2, n)
    else:
        print("Please Check Input")

slopeCompare()
